In [1]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam

In [2]:
data = pd.read_csv('DeltaIS_data.csv')
data

,Company,Time,Total Revenues,Operating Income,Net Income,Revenue Per Share,Basic EPS,Normalized Basic EPS,Dividend Per Share,EBITA,EBIT,Normalized Net Income,Price_Change
0,VERU,2018-09-01,2.2,-9.3,-17.3,-0.10,-0.19,-0.08,0.00,-9.1,-9.3,-7.1,-1.23
1,MAS,2017-12-01,-1347.0,-52.0,40.0,-3.43,0.19,-0.15,0.02,-51.0,-52.0,-62.7,12.32
2,EPD,2014-12-01,224.2,86.1,190.5,-0.75,0.05,-0.03,0.08,91.1,86.1,2.8,2.97
3,ANSS,2015-12-01,6.8,5.1,-2.2,0.36,0.05,0.12,0.00,2.2,5.1,4.7,10.50
4,GLUU,2017-12-01,86.2,-5.2,-10.2,0.59,-0.06,-0.01,0.00,-9.7,-5.2,-3.1,1.70
5,SBUX,2011-10-01,993.0,132.8,300.1,0.63,0.19,0.08,0.10,133.8,132.8,120.0,6.90
6,CRM,2018-01-01,2103.0,236.0,37.0,2.48,0.03,0.22,0.00,297.0,236.0,158.8,34.81
7,TJX,2016-01-01,1866.5,98.2,62.6,1.98,0.09,0.09,0.07,98.2,98.2,57.2,2.65
8,PG,2015-06-01,-3652.0,-833.0,-4607.0,-1.27,-1.69,-0.16,0.14,-890.0,-833.0,-446.6,-0.35
9,WAT,2013-12-01,60.6,-5.6,-11.4,1.30,0.02,0.05,0.00,8.6,-5.6,-4.6,12.88


In [3]:
prices = data['Price_Change'] 
data = data.drop(['Company', 'Time', 'Price_Change'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price_Change', prices)
data

Total Revenues
Done
Operating Income
Done
Net Income
Done
Revenue Per Share
Done
Basic EPS
Done
Normalized Basic EPS
Done
Dividend Per Share
Done
EBITA
Done
EBIT
Done
Normalized Net Income
Done


,Total Revenues,Operating Income,Net Income,Revenue Per Share,Basic EPS,Normalized Basic EPS,Dividend Per Share,EBITA,EBIT,Normalized Net Income,Price_Change
0,0.550790,0.196150,0.480833,0.031883,0.344293,0.165317,0.480023,0.196798,0.196150,0.230765,-1.23
1,0.545027,0.195790,0.481136,0.031669,0.344298,0.165316,0.481181,0.196445,0.195790,0.230071,12.32
2,0.551738,0.196953,0.481932,0.031842,0.344296,0.165319,0.484655,0.197642,0.196953,0.230889,2.97
3,0.550809,0.196271,0.480913,0.031913,0.344296,0.165322,0.480023,0.196893,0.196271,0.230912,10.50
4,0.551149,0.196184,0.480871,0.031928,0.344295,0.165319,0.480023,0.196793,0.196184,0.230815,1.70
5,0.555022,0.197346,0.482512,0.031931,0.344298,0.165321,0.485814,0.198001,0.197346,0.232352,6.90
6,0.559763,0.198215,0.481121,0.032050,0.344296,0.165324,0.480023,0.199375,0.198215,0.232836,34.81
7,0.558753,0.197055,0.481256,0.032018,0.344297,0.165321,0.484076,0.197701,0.197055,0.231568,2.65
8,0.535182,0.189214,0.456557,0.031808,0.344272,0.165316,0.488130,0.189381,0.189214,0.225278,-0.35
9,0.551039,0.196181,0.480865,0.031974,0.344296,0.165320,0.480023,0.196947,0.196181,0.230796,12.88


In [4]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:10] # Parameters
y = NpMatrix[:,10] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))

X: [[0.55078975 0.19614954 0.48083335 ... 0.19679791 0.19614954 0.23076491]
 [0.54502704 0.19579001 0.48113643 ... 0.19644511 0.19579001 0.2300708 ]
 [0.55173795 0.19695281 0.48193246 ... 0.19764158 0.19695281 0.2308885 ]
 ...
 [0.54792847 0.19951753 0.47877213 ... 0.20016419 0.19951753 0.23339155]
 [0.54039705 0.19187471 0.47656652 ... 0.19186468 0.19187471 0.22802217]
 [0.55013796 0.19398813 0.47958932 ... 0.19463483 0.19398813 0.22874749]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 10
Total Number of Training instances: 4500


[-1.23 12.32  2.97 ...  5.11 -1.5  -1.95]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector -1.23
Total number of y values 4500


In [5]:
number_of_features = len(X[1])

#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=100, activation='relu', input_shape=(number_of_features,)))
    
        #Add Dropout Layer
    network.add(layers.Dropout(0.2))

    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=1, activation='tanh'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [6]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 500, 
                                 batch_size=5)

In [7]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 3600 samples
Epoch 1/500
3600/3600 [==============================] - 2s 574us/sample - loss: 15.4709 - mae: 15.4709 - mse: 1780.2393
Epoch 2/500
3600/3600 [==============================] - 2s 457us/sample - loss: 15.4527 - mae: 15.4527 - mse: 1779.4174
Epoch 3/500
3600/3600 [==============================] - 1s 346us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4083
Epoch 4/500
3600/3600 [==============================] - 1s 412us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4043
Epoch 5/500
3600/3600 [==============================] - 1s 319us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4044
Epoch 6/500
3600/3600 [==============================] - 1s 315us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4039
Epoch 7/500
3600/3600 [==============================] - 1s 279us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4062
Epoch 8/500
3600/3600 [==============================] - 1s 271us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4049
Epoch 9/50

Epoch 68/500
3600/3600 [==============================] - 2s 492us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4041
Epoch 69/500
3600/3600 [==============================] - 1s 412us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4043
Epoch 70/500
3600/3600 [==============================] - 2s 545us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4047
Epoch 71/500
3600/3600 [==============================] - 2s 476us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4043
Epoch 72/500
3600/3600 [==============================] - 2s 457us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4042
Epoch 73/500
3600/3600 [==============================] - 2s 424us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4039
Epoch 74/500
3600/3600 [==============================] - 1s 380us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4033
Epoch 75/500
3600/3600 [==============================] - 1s 371us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4042
Epoch 76/500
3600/3600 [

3600/3600 [==============================] - 2s 496us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4050
Epoch 135/500
3600/3600 [==============================] - 2s 443us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4039
Epoch 136/500
3600/3600 [==============================] - 2s 430us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4042
Epoch 137/500
3600/3600 [==============================] - 2s 449us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4044
Epoch 138/500
3600/3600 [==============================] - 1s 391us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4047
Epoch 139/500
3600/3600 [==============================] - 1s 406us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4047
Epoch 140/500
3600/3600 [==============================] - 1s 391us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4036
Epoch 141/500
3600/3600 [==============================] - 1s 369us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4041
Epoch 142/500
3600/3600 [=====

3600/3600 [==============================] - 2s 446us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4041
Epoch 201/500
3600/3600 [==============================] - 1s 400us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4042
Epoch 202/500
3600/3600 [==============================] - 1s 399us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4059
Epoch 203/500
3600/3600 [==============================] - 1s 406us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4041
Epoch 204/500
3600/3600 [==============================] - 2s 434us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4042
Epoch 205/500
3600/3600 [==============================] - 1s 365us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4043
Epoch 206/500
3600/3600 [==============================] - 1s 393us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4052
Epoch 207/500
3600/3600 [==============================] - 1s 416us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4049
Epoch 208/500
3600/3600 [=====

3600/3600 [==============================] - 1s 352us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4036
Epoch 267/500
3600/3600 [==============================] - 1s 410us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4033
Epoch 268/500
3600/3600 [==============================] - 2s 453us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4043
Epoch 269/500
3600/3600 [==============================] - 2s 420us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4050
Epoch 270/500
3600/3600 [==============================] - 1s 399us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4041
Epoch 271/500
3600/3600 [==============================] - 1s 413us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4041
Epoch 272/500
3600/3600 [==============================] - 1s 410us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4050
Epoch 273/500
3600/3600 [==============================] - 1s 368us/sample - loss: 15.4525 - mae: 15.4525 - mse: 1779.4052
Epoch 274/500
3600/3600 [=====

...

3600/3600 [==============================] - 1s 356us/sample - loss: 15.8626 - mae: 15.8626 - mse: 2721.5027
Epoch 201/500
3600/3600 [==============================] - 1s 384us/sample - loss: 15.8608 - mae: 15.8608 - mse: 2721.4585
Epoch 202/500
3600/3600 [==============================] - 1s 378us/sample - loss: 15.8611 - mae: 15.8611 - mse: 2721.4541
Epoch 203/500
3600/3600 [==============================] - 1s 363us/sample - loss: 15.8645 - mae: 15.8645 - mse: 2721.5176
Epoch 204/500
3600/3600 [==============================] - 1s 350us/sample - loss: 15.8608 - mae: 15.8608 - mse: 2721.4363
Epoch 205/500
3600/3600 [==============================] - 1s 352us/sample - loss: 15.8621 - mae: 15.8621 - mse: 2721.4351
Epoch 206/500
3600/3600 [==============================] - 1s 399us/sample - loss: 15.8597 - mae: 15.8597 - mse: 2721.4319
Epoch 207/500
3600/3600 [==============================] - 1s 352us/sample - loss: 15.8606 - mae: 15.8606 - mse: 2721.3108
Epoch 208/500
3600/3600 [=====

3600/3600 [==============================] - 1s 364us/sample - loss: 15.8625 - mae: 15.8625 - mse: 2721.4685
Epoch 267/500
3600/3600 [==============================] - 1s 353us/sample - loss: 15.8606 - mae: 15.8606 - mse: 2721.4578
Epoch 268/500
3600/3600 [==============================] - 1s 398us/sample - loss: 15.8614 - mae: 15.8614 - mse: 2721.3984
Epoch 269/500
3600/3600 [==============================] - 1s 351us/sample - loss: 15.8622 - mae: 15.8622 - mse: 2721.4719
Epoch 270/500
3600/3600 [==============================] - 1s 352us/sample - loss: 15.8623 - mae: 15.8622 - mse: 2721.4487
Epoch 271/500
3600/3600 [==============================] - 1s 353us/sample - loss: 15.8610 - mae: 15.8610 - mse: 2721.3784
Epoch 272/500
3600/3600 [==============================] - 1s 401us/sample - loss: 15.8627 - mae: 15.8627 - mse: 2721.6008
Epoch 273/500
3600/3600 [==============================] - 1s 353us/sample - loss: 15.8626 - mae: 15.8626 - mse: 2721.4075
Epoch 274/500
3600/3600 [=====

3600/3600 [==============================] - 1s 378us/sample - loss: 15.8625 - mae: 15.8625 - mse: 2721.4058
Epoch 333/500
3600/3600 [==============================] - 1s 368us/sample - loss: 15.8621 - mae: 15.8621 - mse: 2721.4448
Epoch 334/500
3600/3600 [==============================] - 2s 470us/sample - loss: 15.8615 - mae: 15.8615 - mse: 2721.4055
Epoch 335/500
3600/3600 [==============================] - 2s 514us/sample - loss: 15.8630 - mae: 15.8630 - mse: 2721.4905
Epoch 336/500
3600/3600 [==============================] - 2s 426us/sample - loss: 15.8612 - mae: 15.8612 - mse: 2721.2927
Epoch 337/500
3600/3600 [==============================] - 1s 410us/sample - loss: 15.8613 - mae: 15.8613 - mse: 2721.4514
Epoch 338/500
3600/3600 [==============================] - 1s 413us/sample - loss: 15.8604 - mae: 15.8604 - mse: 2721.3716
Epoch 339/500
3600/3600 [==============================] - 2s 581us/sample - loss: 15.8614 - mae: 15.8614 - mse: 2721.5017
Epoch 340/500
3600/3600 [=====

3600/3600 [==============================] - 1s 405us/sample - loss: 15.8622 - mae: 15.8622 - mse: 2721.3318
Epoch 399/500
3600/3600 [==============================] - 2s 434us/sample - loss: 15.8596 - mae: 15.8596 - mse: 2721.3689
Epoch 400/500
3600/3600 [==============================] - 1s 368us/sample - loss: 15.8589 - mae: 15.8589 - mse: 2721.2727
Epoch 401/500
3600/3600 [==============================] - 1s 375us/sample - loss: 15.8619 - mae: 15.8619 - mse: 2721.4529
Epoch 402/500
3600/3600 [==============================] - 1s 385us/sample - loss: 15.8614 - mae: 15.8614 - mse: 2721.3210
Epoch 403/500
3600/3600 [==============================] - 1s 380us/sample - loss: 15.8610 - mae: 15.8610 - mse: 2721.3323
Epoch 404/500
3600/3600 [==============================] - 1s 376us/sample - loss: 15.8626 - mae: 15.8626 - mse: 2721.4822
Epoch 405/500
3600/3600 [==============================] - 1s 393us/sample - loss: 15.8602 - mae: 15.8602 - mse: 2721.3911
Epoch 406/500
3600/3600 [=====

3600/3600 [==============================] - 1s 301us/sample - loss: 15.8601 - mae: 15.8601 - mse: 2721.3374
Epoch 465/500
3600/3600 [==============================] - 1s 286us/sample - loss: 15.8617 - mae: 15.8617 - mse: 2721.4458
Epoch 466/500
3600/3600 [==============================] - 1s 274us/sample - loss: 15.8615 - mae: 15.8615 - mse: 2721.4216
Epoch 467/500
3600/3600 [==============================] - 1s 271us/sample - loss: 15.8622 - mae: 15.8622 - mse: 2721.4500
Epoch 468/500
3600/3600 [==============================] - 1s 273us/sample - loss: 15.8604 - mae: 15.8604 - mse: 2721.4014
Epoch 469/500
3600/3600 [==============================] - 1s 285us/sample - loss: 15.8621 - mae: 15.8621 - mse: 2721.4736
Epoch 470/500
3600/3600 [==============================] - 1s 316us/sample - loss: 15.8604 - mae: 15.8604 - mse: 2721.4019
Epoch 471/500
3600/3600 [==============================] - 1s 273us/sample - loss: 15.8626 - mae: 15.8626 - mse: 2721.4600
Epoch 472/500
3600/3600 [=====

In [8]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-17.71327766 -15.2948999  -15.828156   -14.63874426 -16.031914  ]
Mean MAE: -15.901398362583583
Standard Deviation of the mean: 1.0263957446974805
